## BIP 32 Python Implementation

**BIP 32** describes the generation of Hierarchical Deterministic (HD) wallets.

Comprehensive step explanations can be found in the accompanying blog post: [Ethereum 201: HD Wallets](TODO).

_Disclaimer: this code is for educational purposes and not optimized for a production environment._

### Start with a seed
BIP 32 picks up where BIP 39 left off - with a 512-bit seed:

In [1]:
seed = 'fffcf9f6f3f0edeae7e4e1dedbd8d5d2cfccc9c6c3c0bdbab7b4b1aeaba8a5a29f9c999693908d8a8784817e7b7875726f6c696663605d5a5754514e4b484542'

### Derive the master private key and chain code

In [2]:
import binascii
import hmac
import hashlib

# the HMAC-SHA512 `key` and `data` must be bytes:
seed_bytes = binascii.unhexlify(seed)

I = hmac.new(b'Bitcoin seed', seed_bytes, hashlib.sha512).digest()
L, R = I[:32], I[32:]

master_private_key = int.from_bytes(L, 'big')
master_chain_code = R

print(f'master private key (hex): {hex(master_private_key)}')
print(f'master chain code (bytes): {master_chain_code}')

master private key (hex): 0x4b03d6fc340455b363f51020ad3ecca4f0850280cf436c70c727923f6db46c3e
master chain code (bytes): b'`I\x9f\x80\x1b\x89m\x83\x17\x9aCt\xae\xb7\x82*\xae\xac\xea\xa0\xdb\x1f\x85\xee>\x90LM\xef\xbd\x96\x89'


### Derive the root key (extended private key)

In [3]:
import base58

VERSION_BYTES = {
    'mainnet_public': binascii.unhexlify('0488b21e'),
    'mainnet_private': binascii.unhexlify('0488ade4'),
    'testnet_public': binascii.unhexlify('043587cf'),
    'testnet_private': binascii.unhexlify('04358394'),
}

version_bytes = VERSION_BYTES['mainnet_private']
depth_byte = b'\x00'
parent_fingerprint = b'\x00' * 4
child_number_bytes = b'\x00' * 4
key_bytes = b'\x00' + L

all_parts = (
    version_bytes,      # 4 bytes  
    depth_byte,         # 1 byte
    parent_fingerprint, # 4 bytes
    child_number_bytes, # 4 bytes
    master_chain_code,  # 32 bytes
    key_bytes,          # 33 bytes
)

all_bytes = b''.join(all_parts)
root_key = base58.b58encode_check(all_bytes).decode('utf8')
print(f'root key: {root_key}')

root key: xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U


### Some elliptic curve utility functions
You can't escape math.

In [4]:
from ecdsa import SECP256k1
from ecdsa.ecdsa import Public_key

SECP256k1_GEN = SECP256k1.generator

def serialize_curve_point(p):
   x, y = p.x(), p.y()
   if y & 1:
      return b'\x03' + x.to_bytes(32, 'big')
   else:
      return b'\x02' + x.to_bytes(32, 'big')

def curve_point_from_int(k):
   return Public_key(SECP256k1_GEN, SECP256k1_GEN * k).point

### Define a fingerprint function
A fingerprint is four bytes - a link between child and parent keys.

In [5]:
from ecdsa import SECP256k1
from ecdsa.ecdsa import Public_key

SECP256k1_GEN = SECP256k1.generator

def fingerprint_from_priv_key(k):
    K = curve_point_from_int(k)
    K_compressed = serialize_curve_point(K)
    identifier = hashlib.new(
      'ripemd160',
      hashlib.sha256(K_compressed).digest(),
    ).digest()
    return identifier[:4]

### Define the child key derivation function

In [6]:
SECP256k1_ORD = SECP256k1.order

def derive_ext_private_key(private_key, chain_code, child_number):
    if child_number >= 2 ** 31:
        # Generate a hardened key
        data = b'\x00' + private_key.to_bytes(32, 'big')
    else:
        # Generate a non-hardened key
        p = curve_point_from_int(private_key)
        data = serialize_curve_point(p)

    data += child_number.to_bytes(4, 'big')

    hmac_bytes = hmac.new(chain_code, data, hashlib.sha512).digest()
    L, R = hmac_bytes[:32], hmac_bytes[32:]

    L_as_int = int.from_bytes(L, 'big')
    child_private_key = (L_as_int + private_key) % SECP256k1_ORD
    child_chain_code = R

    return (child_private_key, child_chain_code)

### Run the child key derivation function once per path depth
We're deriving keys for the account at the "default" path: `m/44'/60'/0'/0/0`.

In [7]:
# Break each depth into integers (m/44'/60'/0'/0/0)
#    e.g. (44, 60, 0, 0, 0)
# If hardened, add 2*31 to the number:
#    e.g. (2**31 + 44, 2**31 + 60, 2**31 + 0, 0, 0)
path_numbers = (2147483692, 2147483708, 2147483648, 0, 0)

depth = 0
parent_fingerprint = None
child_number = None
private_key = master_private_key
chain_code = master_chain_code

for i in path_numbers:
    depth += 1
    print(f"depth: {depth}")
    
    child_number = i
    print(f"child_number: {child_number}")
    
    parent_fingerprint = fingerprint_from_priv_key(private_key)
    print(f"parent_fingerprint: {parent_fingerprint}")
    
    private_key, chain_code = derive_ext_private_key(private_key, chain_code, i)
    print(f"private_key: {private_key}")
    print(f"chain_code: {chain_code}\n")

depth: 1
child_number: 2147483692
parent_fingerprint: b'\xbd\x16\xbe\xe5'
private_key: 61609765036830346750313247117170114938513374360478483024968542854420611267661
chain_code: b']\xf6\xf8\xa7\xd2\xa2\xea\xc1C\x01\xa1\x90\xdf dn\xfe\x11G\xabW\xb51Q\xa6 \x1b\xc0rW\xcf\x19'

depth: 2
child_number: 2147483708
parent_fingerprint: b'G\x89\x1eB'
private_key: 15393795590512331372998570934590381713921834558194848920654607994440388065560
chain_code: b'\x05\\F\xe1U\xf2\x04a\x9b\x14\xbe\x00\x88\xe6\x95|0*\xd5$\xbbu\xea~\\\xdd\xb2\x82\x8b\xddC\x99'

depth: 3
child_number: 2147483648
parent_fingerprint: b'M\xd6\x18\x80'
private_key: 19102194832216337410531172758576291606475083221159356381572566811711190190745
chain_code: b'6t5\x17\xf58\xa4S\xbe\xc7\xd9\xd04Goa\x9bC\x93\xcf\\\xd03\xaa\xac\xbd\xde\x90\xbdS\xdf('

depth: 4
child_number: 0
parent_fingerprint: b'C\xed\x0b-'
private_key: 40085757126754576363899103523271451610806901886905148899863161166595106319204
chain_code: b'I\xcc\xe3+\xed\x1f^\xf6\xe

### Derive the extended private key

In [8]:
version_bytes = VERSION_BYTES['mainnet_private']
depth_byte = depth.to_bytes(1, 'big')
child_number_bytes = child_number.to_bytes(4, 'big')
key_bytes = b'\x00' + private_key.to_bytes(32, 'big')

all_parts = (
    version_bytes,      # 4 bytes  
    depth_byte,         # 1 byte
    parent_fingerprint, # 4 bytes
    child_number_bytes, # 4 bytes
    chain_code,         # 32 bytes
    key_bytes,          # 33 bytes
)
all_bytes = b''.join(all_parts)
extended_private_key = base58.b58encode_check(all_bytes).decode('utf8')
print(f'xprv: {extended_private_key}')

xprv: xprvA2vDkmMuK1Ae2eF92xyQpn6qZzHoGTnV5hXrBw7UExUTXeMFTZDLF7cRD6vhR785RMF2EC6mAo3ojRqFEUU8VxTSzGq1jvmXSBTxoCGSSVG


**Nailed it!** Now, for more practical purposes, we'll need a public address and a private key.

### Display the private key

In [9]:
print(f'private key: {hex(private_key)}')

private key: 0x3c4cf049f83a5870ab31c396a0d46783c3e3974da1364ea5a2477548d36b5f8f


### Derive the public key

In [10]:
# Derive the public key Point:
p = curve_point_from_int(private_key)
print(f'Point object: {p}\n')

# Serialize the Point, p
public_key_bytes = serialize_curve_point(p)

print(f'public key (hex): 0x{public_key_bytes.hex()}')

Point object: (34628879175116161227789129351591737524694652815106357809683939650023911982126,16686316349534243923155184728961992244162372383889866453776214730676940635074)

public key (hex): 0x024c8f4044470bd42b81a8b233e2f954b63f4ee2c32c8d44288b44188754e2042e


### Derive the public address

In [11]:
from eth_utils import keccak

# Hash the concatenated x and y public key point values:
digest = keccak(p.x().to_bytes(32, 'big') + p.y().to_bytes(32, 'big'))

# Take the last 20 bytes and add '0x' to the front:
address = '0x' + digest[-20:].hex()

print(f'address: {address}')

address: 0xbbec2620cb01adae3f96e1fa39f997f06bfb7ca0


# 🚀